In [1]:
# if you cloned diffop_factorization module in your Documents folder, running the following line allows your Sage
# to know where look for this module
import sys; sys.path.append('/~/Documents')

In [2]:
from diffop_factorization import invariant_subspace

## Generic examples

In [3]:
mat1 = MatrixSpace(CC, 3).random_element().change_ring(CBF)
mat2 = MatrixSpace(CC, 3).random_element().change_ring(CBF)

(invariant_subspace([mat1]), invariant_subspace([mat1, mat2]))

([([1.000000 +/- 3.10e-7] + [+/- 3.10e-7]*I, [-0.204006 +/- 6.26e-7] + [-0.459831 +/- 2.96e-7]*I, [-0.012579 +/- 4.61e-7] + [-0.050031 +/- 3.76e-7]*I)],
 None)

## Two matrices with a commun eigenvector

In [4]:
C = ComplexBallField(100)
mat1 = MatrixSpace(CC, 3).random_element().change_ring(C)
mat2 = MatrixSpace(CC, 3).random_element().change_ring(C)
mat1[1:3,0] , mat2[1:3,0] = MatrixSpace(C, 2, 1).zero(), MatrixSpace(C, 2, 1).zero()
ran = MatrixSpace(CC, 3).random_element().change_ring(C)
mat1, mat2 = ~ran * mat1 * ran, ~ran * mat2 * ran

V = invariant_subspace([mat1, mat2]); V

[([1.000000000000000000 +/- 3.35e-19] + [+/- 3.35e-19]*I, [0.661623722295726314 +/- 5.40e-19] + [0.186930227269756273 +/- 9.15e-19]*I, [-0.492541207235549876 +/- 7.62e-19] + [-0.406966757051311410 +/- 4.73e-19]*I)]

In [5]:
v = V[0]
v1, v2 = mat1*v, mat2*v
v/v[0], v1/v1[0], v2/v2[0]

(([1.000000000000000000 +/- 6.69e-19] + [+/- 6.69e-19]*I, [0.661623722295726314 +/- 8.24e-19] + [0.18693022726975627 +/- 3.26e-18]*I, [-0.49254120723554988 +/- 5.07e-18] + [-0.406966757051311410 +/- 7.74e-19]*I),
 ([1.000000000000000 +/- 9.40e-18] + [+/- 9.40e-18]*I, [0.6616237222957263 +/- 2.15e-17] + [0.1869302272697563 +/- 3.51e-17]*I, [-0.4925412072355499 +/- 3.27e-17] + [-0.40696675705131141 +/- 8.42e-18]*I),
 ([1.0000000000000000 +/- 2.25e-18] + [+/- 2.25e-18]*I, [0.66162372229572631 +/- 5.92e-18] + [0.18693022726975627 +/- 4.55e-18]*I, [-0.49254120723554988 +/- 6.52e-18] + [-0.40696675705131141 +/- 2.23e-18]*I))

## Matrices with a prescribed dimension of invariant subspace

In [6]:
def test (td, pd, prec, N) :
    
    """ td = total dimension,
        pd = prescribed dimension,
        prec = precision,
        N = number of matrices """

    C = ComplexBallField(prec)
    
    Mats = []
    for cpt in range(N):
        mat = MatrixSpace(CC, td).random_element().change_ring(C)
        mat[pd:,:pd] = MatrixSpace(C, td-pd, pd).zero()
        Mats.append(mat)
    
    ran = MatrixSpace(CC, td).random_element().change_ring(C)
    for j, mat in enumerate(Mats):
        Mats[j] = ~ran * mat * ran
    
    V = invariant_subspace(Mats)
    if V is None:
        print('cc')
    return pd == len(V)

dim_max = 20
%time all(test(dim, dim//3, 1000, 5) for dim in [4,6..dim_max]) # 10sec for dim_max = 20

CPU times: user 9.7 s, sys: 4.08 ms, total: 9.7 s
Wall time: 9.7 s


True

## Tricky case of MeatAxe algorithm

In [7]:
bN = 2         # number of blocs
bdim = 3       # dimension of blocs
N = 2          # number of matrices
dim = bdim*bN  # total dimension
prec = 200
C = ComplexBallField(prec)

Mats = []
for cpt in range(N):
    A = MatrixSpace(CC, bdim).random_element().change_ring(C)
    mat = block_matrix(C, [[A.parent().zero()]*i+[A]+[A.parent().random_element()\
                                                      for j in range(bN-i-1)] for i in range(bN)])
    Mats.append(mat)

ran = MatrixSpace(CC, dim).random_element().change_ring(C)
Mats = [~ran * mat * ran for mat in Mats]

V = invariant_subspace(Mats)
len(V)

3

## When a linear combination is not enough

In [8]:
prec = 100
C = ComplexBallField(prec)

mat1 = matrix(C, [[0,1,0],[0,0,1],[0,0,0]])
mat2 = matrix(C, [[0,0,0],[1,0,0],[0,-1,0]])
ran = MatrixSpace(CC, 3).random_element().change_ring(C)
mat1, mat2 = ~ran * mat1 * ran, ~ran * mat2 * ran

invariant_subspace([mat1, mat2], verbose=True) is None

The partition is currently:  [3]
Lines checked
Need to check nolines.
Need to compute a basis of the algebra


True